In [5]:
// --- COMANDOS MÁGICOS DE EVCXR (Notebook) ---
// Aquí definimos las dependencias directamente en la celda
:dep reqwest = { version = "0.11", features = ["json", "blocking"] }
:dep tokio = { version = "1", features = ["full"] }
:dep serde = { version = "1.0", features = ["derive"] }
:dep serde_json = "1.0"
:dep prettytable-rs = "^0.10"

// Importaciones
use reqwest::Client;
use serde::Deserialize;
use std::time::{Instant, Duration};
use std::thread;
use prettytable::{Table, Row, Cell, format};

println!("✅ Dependencias cargadas y listas para la acción, mi vida.");

✅ Dependencias cargadas y listas para la acción, mi vida.


In [6]:
// --- ESTRUCTURAS DE DATOS ---

#[derive(Deserialize, Debug)]
struct BinanceTicker {
    symbol: String,
    bidPrice: String, 
    askPrice: String,
}

#[derive(Deserialize, Debug)]
struct OkxTickerData {
    instId: String,
    bidPx: String,
    askPx: String,
}

#[derive(Deserialize, Debug)]
struct OkxResponse {
    data: Vec<OkxTickerData>,
}

println!("✅ Estructuras definidas. El compilador ya sabe qué esperar de las APIs.");

✅ Estructuras definidas. El compilador ya sabe qué esperar de las APIs.


In [ ]:
// --- CONFIGURACIÓN ---
const SYMBOL_BINANCE: &str = "BTCUSDT";
const SYMBOL_OKX: &str = "BTC-USDT";
const FEE_TAKER: f64 = 0.001; // 0.1%

// Creamos un Runtime de Tokio explícito para poder ejecutar 'await' en el notebook
let rt = tokio::runtime::Runtime::new().unwrap();

// Ejecutamos la lógica dentro del runtime
rt.block_on(async {
    println!("--- INICIANDO ANÁLISIS DE VIABILIDAD (NOTEBOOK EDITION) ---");
    
    let client = Client::new();
    let mut table = Table::new();
    table.set_format(*format::consts::FORMAT_NO_LINESEP_WITH_TITLE);
    table.set_titles(Row::new(vec![
        Cell::new("Latencia Binance"),
        Cell::new("Latencia OKX"),
        Cell::new("Spread B->O"),
        Cell::new("Spread O->B"),
        Cell::new("Rentable?"),
    ]));

    let mut total_lat_b = 0.0;
    let mut total_lat_o = 0.0;
    let iterations = 5;

    println!("📡 Capturando datos en tiempo real...");

    for _ in 0..iterations {
        // 1. BINANCE
        let start_b = Instant::now();
        let url_b = format!("https://api.binance.com/api/v3/ticker/bookTicker?symbol={}", SYMBOL_BINANCE);
        // Usamos un match para manejar errores de red sin colapsar el notebook
        let res_b = client.get(&url_b).send().await;
        
        // Si falla la red, saltamos esta iteración
        if res_b.is_err() { println!("Error conectando a Binance"); continue; }
        
        let ticker_b: BinanceTicker = res_b.unwrap().json().await.unwrap();
        let lat_b = start_b.elapsed().as_secs_f64() * 1000.0;
        
        // Parsing
        let b_bid: f64 = ticker_b.bidPrice.parse().unwrap_or(0.0);
        let b_ask: f64 = ticker_b.askPrice.parse().unwrap_or(0.0);

        // 2. OKX
        let start_o = Instant::now();
        let url_o = format!("https://www.okx.com/api/v5/market/ticker?instId={}", SYMBOL_OKX);
        let res_o = client.get(&url_o).send().await;
        
        if res_o.is_err() { println!("Error conectando a OKX"); continue; }

        let json_o: OkxResponse = res_o.unwrap().json().await.unwrap();
        let lat_o = start_o.elapsed().as_secs_f64() * 1000.0;

        let ticker_o = &json_o.data[0];
        let o_bid: f64 = ticker_o.bidPx.parse().unwrap_or(0.0);
        let o_ask: f64 = ticker_o.askPx.parse().unwrap_or(0.0);

        // Acumuladores
        total_lat_b += lat_b;
        total_lat_o += lat_o;

        // CÁLCULOS
        let costo_b = b_ask * (1.0 + FEE_TAKER);
        let venta_o = o_bid * (1.0 - FEE_TAKER);
        let profit_1 = ((venta_o - costo_b) / costo_b) * 100.0;

        let costo_o = o_ask * (1.0 + FEE_TAKER);
        let venta_b = b_bid * (1.0 - FEE_TAKER);
        let profit_2 = ((venta_b - costo_o) / costo_o) * 100.0;

        let rentable = if profit_1 > 0.0 || profit_2 > 0.0 { "SÍ" } else { "NO" };

        table.add_row(Row::new(vec![
            Cell::new(&format!("{:.1}ms", lat_b)),
            Cell::new(&format!("{:.1}ms", lat_o)),
            Cell::new(&format!("{:.4}%", profit_1)),
            Cell::new(&format!("{:.4}%", profit_2)),
            Cell::new(rentable),
        ]));

        // Pequeña pausa para no saturar APIs (respeto al rate limit)
        tokio::time::sleep(Duration::from_millis(800)).await;
    }

    // IMPRIMIR RESULTADOS
    println!("\n--- RESULTADOS ---");
    table.printstd();

    let avg_b = total_lat_b / iterations as f64;
    let avg_o = total_lat_o / iterations as f64;
    
    println!("\n📊 Resumen de Latencia:");
    println!("Binance Avg: {:.2} ms", avg_b);
    println!("OKX Avg:     {:.2} ms", avg_o);
    
    if avg_b > 200.0 || avg_o > 200.0 {
        println!("⚠️ La latencia es alta para arbitraje de alta frecuencia.");
    } else {
        println!("🚀 Latencia saludable. Sistema listo para optimización.");
    }
});

--- INICIANDO ANÁLISIS DE VIABILIDAD (NOTEBOOK EDITION) ---
📡 Capturando datos en tiempo real...

--- RESULTADOS ---
+------------------+--------------+-------------+-------------+-----------+
| Latencia Binance | Latencia OKX | Spread B->O | Spread O->B | Rentable? |
+------------------+--------------+-------------+-------------+-----------+
| 447.5ms          | 452.0ms      | -0.2062%    | -0.1935%    | NO        |
| 231.0ms          | 324.0ms      | -0.2062%    | -0.1935%    | NO        |
| 272.9ms          | 282.3ms      | -0.2020%    | -0.1977%    | NO        |
| 250.6ms          | 372.9ms      | -0.2020%    | -0.1977%    | NO        |
| 234.5ms          | 311.6ms      | -0.2051%    | -0.1946%    | NO        |
+------------------+--------------+-------------+-------------+-----------+

📊 Resumen de Latencia:
Binance Avg: 287.28 ms
OKX Avg:     348.57 ms
⚠️ La latencia es alta para arbitraje de alta frecuencia.
